<img src='img/logo.png'>
<img src='img/title.png'>

# Preprocessing

# Table of Contents
* [Preprocessing](#Preprocessing)
	* [Setup](#Setup)
	* [Example workflow with `MinMaxScaler`](#Example-workflow-with-MinMaxScaler)
	* [Scaling training and test data the same way](#Scaling-training-and-test-data-the-same-way)
	* [The effect of preprocessing on supervised learning](#The-effect-of-preprocessing-on-supervised-learning)
		* [`preprocessing.scale`](#preprocessing.scale)
* [Summary](#Summary)


## Setup

In [ ]:
#Fix working directory
%cd notebooks

In [ ]:
import numpy as np
import pandas as pd

import holoviews as hv
import panel as pn
hv.extension('bokeh')

%matplotlib inline
import matplotlib.pyplot as plt
plt.rcParams['image.interpolation'] = "none"
np.set_printoptions(precision=3)
plt.rcParams['image.cmap'] = "gray"

import src.mglearn as mglearn

http://scikit-learn.org/stable/modules/classes.html#module-sklearn.preprocessing

Preprocessing is often desirable for the following reasons:
 * Input data may have many dimensions and/or colinear dimensions, making it desirable to simplify the fitting problem by reducing the number of columns (features)
 * Input data may not have the statistical properties that are ideal for fitting

The following cell shows how several different preprocessing scalers affect input feature statstics.  An overview of the [sklearn.preprocessing api can be found here](http://scikit-learn.org/stable/modules/classes.html#module-sklearn.preprocessing).

The block beginning with:
```

for scaler in [StandardScaler(), RobustScaler(),
               MinMaxScaler(), Normalizer(norm='l2')]
```

is looping over 4 different scalers:
 * `StandardScaler`: [Remove mean and create unit variance](http://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.StandardScaler.html)
 * `RobustScaler`: [Scaling robust to outliers in input data](http://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.RobustScaler)
 * `MinMaxScaler`: [Scale to the min / max range of each feature](http://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.MinMaxScaler.html#sklearn.preprocessing.MinMaxScaler)
 * `Normalizer(norm='l2')`: [Normalize to a unit norm](http://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.Normalizer.html#sklearn.preprocessing.Normalizer)

In [ ]:
import numpy as np
from sklearn.datasets import make_blobs
from sklearn.preprocessing import StandardScaler, MinMaxScaler, Normalizer, RobustScaler


X, y = make_blobs(n_samples=50, centers=2, random_state=4, cluster_std=1)

# move away from origin
X += 3

# add an outlier
X = np.append(X, [[17,11.5]], axis=0)
y = np.append(y, 1)

scaled = []
for scaler in [StandardScaler(), RobustScaler(),
               MinMaxScaler(), Normalizer(norm='l2')]:
    scaled_X = scaler.fit_transform(X)
    scaled.append((scaler.__class__.__name__, scaled_X))

In [ ]:
plot_options=dict(color='Category', cmap='Category10', size=5)

original = (hv.Points((X[:,0],X[:,1],y), vdims=['Category'])
            .options(**plot_options, xlim=(-18,18), ylim=(-12,12), title='Original Data', width=450, height=450)
           )

scaled_plots = []
for name, scaled_X in scaled:
    p = (hv.Points((scaled_X[:,0],scaled_X[:,1],y), vdims=['Category'])
         .options(**plot_options, xlim=(-3,3), ylim=(-3,3), title=name, width=250, height=250)
        )
    
    scaled_plots.append(p)

scaled_plots = pn.Column(pn.Row(*scaled_plots[:2]), pn.Row(*scaled_plots[2:]))
pn.Row(original, scaled_plots)

## Example workflow with `MinMaxScaler`

This example uses a single training data / test data split, with a `MinMaxScaler` putting features in the `(0, 1)` range.

In [ ]:
from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split
cancer = load_breast_cancer()

X_train, X_test, y_train, y_test = train_test_split(cancer.data, cancer.target,
                                                    random_state=1)
print(X_train.shape)
print(X_test.shape)

In [ ]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()

In [ ]:
scaler.fit(X_train)    # first .fit finds stats needed for scaling

In [ ]:
# Stats of the data before / after MinMaxScaler
X_train_scaled = scaler.transform(X_train)
print("transformed shape: %s" % (X_train_scaled.shape,))
print("per-feature minimum before scaling:\n %s" % X_train.min(axis=0))
print("per-feature maximum before scaling:\n %s" % X_train.max(axis=0))
print("per-feature minimum after scaling:\n %s" % X_train_scaled.min(axis=0))
print("per-feature maximum after scaling:\n %s" % X_train_scaled.max(axis=0))

In [ ]:
# transform test data
X_test_scaled = scaler.transform(X_test)
# print test data properties after scaling
print("per-feature minimum after scaling: %s" % X_test_scaled.min(axis=0))
print("per-feature maximum after scaling: %s" % X_test_scaled.max(axis=0))

## Scaling training and test data the same way

Make sure to call the scaler's `.fit` method separately for training and test data, so the data are not scaled by a different data set's statistics.

In [ ]:
from sklearn.datasets import make_blobs
# make synthetic data
X, _ = make_blobs(n_samples=25, centers=5, random_state=5, cluster_std=2)
# split it into training and test set
X_train, X_test = train_test_split(X, random_state=5, test_size=.1)

In [ ]:
# scale the data using MinMaxScaler
scaler = MinMaxScaler()
scaler.fit(X_train)

X_train_scaled = scaler.transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [ ]:
# rescale the test set separately, so that test set min is 0 and test set max is 1
# DO NOT DO THIS! For illustration purposes only
test_scaler = MinMaxScaler()

test_scaler.fit(X_test)
X_test_scaled_badly = test_scaler.transform(X_test)

In [ ]:
plot_options = dict(size=6, padding=0.5, legend_position='top_right', height=330, width=330)

unscaled = hv.Points(X_train, label='Training Set').options(**plot_options) * \
           hv.Points(X_test, label='Test Set').options(**plot_options, title='Unscaled')

correct_scaling = hv.Points(X_train_scaled, label='Training Set').options(**plot_options) * \
                  hv.Points(X_test_scaled, label='Test Set').options(**plot_options, title='Correct')

bad_scaling = hv.Points(X_train_scaled, label='Training Set').options(**plot_options) * \
              hv.Points(X_test_scaled_badly, label='Test Set').options(**plot_options, title='Incorrect')

pn.Row(unscaled, correct_scaling, bad_scaling)


## The effect of preprocessing on supervised learning

The cells below show that min / max scaling or standardization can improve the predictive value of a support vector classifier for the cancer data set.

In [ ]:
from sklearn.svm import SVC

X_train, X_test, y_train, y_test = train_test_split(cancer.data, cancer.target,
                                                    random_state=0)

svm = SVC(C=100)
svm.fit(X_train, y_train)
print(svm.score(X_test, y_test))

In [ ]:
# preprocessing using 0-1 scaling
scaler = MinMaxScaler()
scaler.fit(X_train)
X_train_scaled = scaler.transform(X_train)
X_test_scaled = scaler.transform(X_test)

# learning an SVM on the scaled training data
svm.fit(X_train_scaled, y_train)
# scoring on the scaled test set

svm.score(X_test_scaled, y_test)

In [ ]:
# preprocessing using zero mean and unit variance scaling
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
scaler.fit(X_train)
X_train_scaled = scaler.transform(X_train)
X_test_scaled = scaler.transform(X_test)

# learning an SVM on the scaled training data
svm.fit(X_train_scaled, y_train)
# scoring on the scaled test set
svm.score(X_test_scaled, y_test)

### `preprocessing.scale`

Scaling with function instead of a class.  The default behavior removes the mean and scales standard deviation to 1.

In [ ]:
from sklearn import preprocessing
import numpy as np
train = np.c_[np.random.normal(10, 2, 100),
              np.random.normal(size=100),
              np.random.normal(0, 5, 100)]

# with_mean/with_std True/False (default True)
scaled = preprocessing.scale(train)
print(np.allclose(0.0, scaled.mean(axis=0)), np.allclose(1.0, scaled.std(axis=0)))

# Summary

In this notebook, we reviewed the following topics in preparation for more advanced topics:

* [Preprocessing](#Preprocessing)
* [Example workflow with `MinMaxScaler`](#Example-workflow-with-MinMaxScaler)
* [Scaling training and test data the same way](#Scaling-training-and-test-data-the-same-way)
* [The effect of preprocessing on supervised learning](#The-effect-of-preprocessing-on-supervised-learning)
* [`preprocessing.scale`](#preprocessing.scale)


<a href='Scaling_and_Normalization_Exercises.ipynb' class='btn btn-primary btn-lg'>Exercises</a>

<img src='img/copyright.png'>